### Balance

In [17]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from punisher.common import *
import qgrid

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
b = Balance()
print("Currencies",  b.currencies)
print("Entries", b)
b.add_currency(c.ETH)
b.update(c.ETH, delta_free=1.0, delta_used=0.0)
print("hi"); print(b.get(c.ETH))
b.get(c.ETH)[BalanceType.FREE]
Balance.from_dict(b.to_dict())

Currencies ['BTC']
Entries {
    "BTC": {
        "free": 1.0,
        "used": 0.0,
        "total": 0.0
    },
    "free": {
        "BTC": 1.0
    },
    "used": {
        "BTC": 0.0
    },
    "total": {
        "BTC": 0.0
    }
}
hi
{<BalanceType.FREE: 'free'>: 1.0, <BalanceType.USED: 'used'>: 0.0, <BalanceType.TOTAL: 'total'>: 1.0}


{
    "BTC": {
        "free": 1.0,
        "used": 0.0,
        "total": 0.0
    },
    "ETH": {
        "free": 1.0,
        "used": 0.0,
        "total": 1.0
    },
    "free": {
        "BTC": 1.0,
        "ETH": 1.0
    },
    "used": {
        "BTC": 0.0,
        "ETH": 0.0
    },
    "total": {
        "BTC": 0.0,
        "ETH": 1.0
    }
}

### Exchange

In [ ]:
# Public informaiton
exchanges = [c.PAPER, c.BINANCE, c.GDAX]#, c.POLONIEX]
a = Asset(c.ETH, c.BTC)
for ex in exchanges:
    print("Exchange", ex)
    exchange = load_exchange(ex)
    print(exchange.timeframes)
    exchange.get_markets()
    exchange.fetch_ohlcv(a, Timeframe.ONE_MIN.value['id'])
    exchange.fetch_order_book(a)
    exchange.fetch_public_trades(a)
    exchange.fetch_ticker(a)

In [ ]:
# Account Information
exchanges = [c.PAPER, c.BINANCE, c.GDAX]
a = Asset(c.ETH, c.BTC)
for ex in exchanges:
    print("Exchange", ex)
    exchange = load_exchange(ex)
    b = exchange.fetch_balance()
    print(exchange.fetch_balance())
    print(exchange.fetch_orders(a))
    print(exchange.fetch_open_orders(a))
    print(exchange.fetch_closed_orders(a))

In [23]:
exchange = load_exchange(c.PAPER) # c.BINANCE
a = Asset(c.ETH, c.BTC)

# Market BUY
print("Exchange", exchange.id)
order_dict = exchange.create_market_buy_order(a, .01)
balance = exchange.fetch_balance()
print(a.base, balance.get(a.base), a.quote, balance.get(a.quote))
print(order_dict)

{'BTC': {'free': 1.0, 'used': 0.0, 'total': 1.0}, 'free': {'BTC': 1.0}, 'used': {'BTC': 0.0}, 'total': {'BTC': 1.0}}
Exchange paper


ExchangeNotAvailable: binance GET https://api.binance.com/api/v1/exchangeInfo HTTPSConnectionPool(host='api.binance.com', port=443): Max retries exceeded with url: /api/v1/exchangeInfo (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x116aaef98>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',)) 

In [ ]:
# Check Order
balance = exchange.fetch_balance()
print(a.base, balance.get(a.base), a.quote, balance.get(a.quote))
found_order = exchange.fetch_order(order_dict['id'], a.symbol)
print("Placed order", found_order) 

In [ ]:
# Sell all remaining quantity
curr_balance = exchange.fetch_balance().get(a.base)
print(a.base, curr_balance[BalanceType.TOTAL])
order = exchange.create_market_sell_order(a, curr_balance[BalanceType.TOTAL])
print(order)

In [ ]:
print(exchange.fetch_balance().get(a.base), exchange.fetch_balance().get(a.quote))

### OHLCV Data

In [24]:
base = c.ETH
quote = c.BTC
exchange = load_exchange(c.BINANCE)
asset = Asset(base, quote)
assets = [ Asset(coin, quote) for coin in [c.ETH, c.LTC] ]
period = Timeframe.ONE_MIN
start = datetime.datetime.utcnow() - datetime.timedelta(hours=2)
end = datetime.datetime.utcnow() - datetime.timedelta(hours=0)

In [ ]:
# Single Coin
df = ohlcv.fetch_and_save_ohlcv_data(exchange, asset, period.value['id'], start, end)
df.head()

In [ ]:
# Load from File
fpath = ohlcv.get_price_data_fpath(asset, exchange.id, period.value['id'])
df = ohlcv.load_chart_data_from_file(fpath)

In [ ]:
# Multiple Coins
ohlcv.download_chart_data(exchange, assets, period.value['id'], start, end)
df = ohlcv.load_multiple_assets(exchange.id, assets, period.value['id'], start)
df.head()

### DataStore

In [25]:
experiment_name = 'mystrategy1'
fname = 'multiasset'
store = FileStore(os.path.join(cfg.DATA_DIR, experiment_name))

In [26]:
# DataFrame --> CSV
store.df_to_csv(df, fname)
df = store.csv_to_df(fname, index='time_epoch')
df.head()

NameError: name 'df' is not defined

In [ ]:
# DataFrame --> JSON
store.df_to_json(df, fname)
df = store.json_to_df(fname, index='time_epoch')
df.head()

In [ ]:
# JSON
dct = {
    'sample': 1,
    'time': datetime.datetime.utcnow()
}
store.save_json(fname, dct)
dct = store.load_json(fname)
dct

### Data Feed

In [29]:
def get_test_live_feed(exchange_id):
    exchange = load_exchange(exchange_id)
    asset = Asset(c.ETH, c.BTC)
    period = Timeframe.ONE_MIN
    start = datetime.datetime.utcnow() - datetime.timedelta(hours=2)
    end = datetime.datetime.utcnow() - datetime.timedelta(hours=0)
    feed_fpath = ohlcv.get_price_data_fpath(asset, exchange.id, period.value['id'])
    feed = ExchangeDataFeed(exchange, [asset], period, feed_fpath, start, end)
    feed.initialize()
    return feed

def get_test_csv_feed(exchange_id):
    start = datetime.datetime(year=2018, month=1, day=9)
    end = datetime.datetime(year=2018, month=1, day=12)
    asset = Asset(c.ETH, c.BTC)
    period = Timeframe.THIRTY_MIN
    feed_fpath = ohlcv.get_price_data_fpath(asset, exchange_id, period.value['id'])
    print(feed_fpath)
    feed = CSVDataFeed(feed_fpath)
#     config = { 
#         "data_provider": PaperExchangeDataProvider(feed),
#         "balance" : Balance()
#     }
    exchange = load_exchange(exchange_id)
    ohlcv.fetch_and_save_ohlcv_data(exchange, asset, period.value['id'], start, end)
    feed.initialize()
    return feed

In [30]:
# CSV Feed
csv_feed = get_test_csv_feed(c.PAPER)

# Grab 1 row at a time
for i in range(1):
    data = csv_feed.next()
    print(data['time_utc'], data['close'])    
    
# Access all rows in history
csv_feed.history().head()

/Users/cfortuner/Dropbox/punisher/paper_ETH_BTC_30m.csv
{'BTC': {'free': 1.0, 'used': 0.0, 'total': 1.0}, 'free': {'BTC': 1.0}, 'used': {'BTC': 0.0}, 'total': {'BTC': 1.0}}
Downloading: ETH/BTC


ExchangeNotAvailable: binance GET https://api.binance.com/api/v1/exchangeInfo HTTPSConnectionPool(host='api.binance.com', port=443): Max retries exceeded with url: /api/v1/exchangeInfo (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x1158c1f60>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',)) 

In [70]:
# Live Feed
live_feed = get_test_live_feed(c.BINANCE)

# Grab 1 row at a time (don't refresh data)
for i in range(3):
    data = live_feed.next(refresh=False)
    print(data['time_utc'], data['close'])

# Refresh data before next query
data = live_feed.next(refresh=True)
print(data['time_utc'], data['close'])
    
# Access History
live_feed.history(t_minus=3).tail()

Loading feed: /Users/cfortuner/Dropbox/punisher/binance_ETH_BTC_1m.csv
Downloading: ETH/BTC
Downloaded rows: 121
2018-01-13 18:33:00 0.099447
2018-01-13 18:34:00 0.099197
2018-01-13 18:35:00 0.099225
Downloading: ETH/BTC
Downloaded rows: 118
2018-01-13 18:36:00 0.099344


,open,high,low,close,volume,time_utc
time_epoch,,,,,,
1515875400,0.099496,0.099596,0.099417,0.099596,155.017,2018-01-13 20:30:00
1515875460,0.099532,0.099712,0.099502,0.099611,211.379,2018-01-13 20:31:00
1515875520,0.099711,0.099800,0.099700,0.099799,135.241,2018-01-13 20:32:00


### OrderType

In [71]:
assert OrderType.from_type_side('limit','buy') == OrderType.LIMIT_BUY
assert OrderType.from_type_side('limit','sell') == OrderType.LIMIT_SELL
assert OrderType.from_type_side('market','buy') == OrderType.MARKET_BUY
assert OrderType.from_type_side('market','sell') == OrderType.MARKET_SELL

In [72]:
assert OrderType.LIMIT_BUY in OrderType.buy_types()
assert OrderType.MARKET_BUY in OrderType.buy_types()
assert OrderType.LIMIT_SELL in OrderType.sell_types()
assert OrderType.MARKET_SELL in OrderType.sell_types()

In [73]:
assert OrderType.LIMIT_BUY.is_buy()
assert OrderType.MARKET_BUY.is_buy()
assert OrderType.LIMIT_SELL.is_sell()
assert OrderType.MARKET_SELL.is_sell()
OrderType.LIMIT_BUY.name, OrderType.LIMIT_BUY.value

('LIMIT_BUY', {'desc': '', 'side': 'buy', 'type': 'limit'})

### Order

In [ ]:
asset = Asset(c.LTC, c.USDT)
order = Order(
    exchange_id=c.PAPER, 
    asset=asset,
    price=250., 
    quantity=1, 
    order_type=OrderType.LIMIT_BUY
)
Order.from_dict(order.to_dict())

### Order Manager

In [31]:
asset = Asset(c.LTC, c.BTC)
exchange = load_exchange(c.PAPER)
exchange.balance = Balance(c.BTC, 5.0)
o1 = Order(exchange.id, asset, price=.1, quantity=1.0, order_type=OrderType.LIMIT_BUY)
o2 = Order(exchange.id, asset, price=.1, quantity=1.0, order_type=OrderType.LIMIT_SELL)
orders = {
    o1.id: o1,
    o2.id: o2
}
orders,exchange.balance

{'BTC': {'free': 1.0, 'used': 0.0, 'total': 1.0}, 'free': {'BTC': 1.0}, 'used': {'BTC': 0.0}, 'total': {'BTC': 1.0}}


({'5b21ac9d066a4f81ba88f03421cef343': {
      "id": "5b21ac9d066a4f81ba88f03421cef343",
      "exchange_id": "paper",
      "exchange_order_id": null,
      "asset": "LTC/BTC",
      "price": 0.1,
      "quantity": 1.0,
      "filled_quantity": 0.0,
      "order_type": "LIMIT_BUY",
      "status": "CREATED",
      "created_time": "2018-01-15T01:16:07.063091",
      "opened_time": null,
      "filled_time": null,
      "canceled_time": null,
      "fee": {},
      "retries": 0
  }, 'b6b0c80f07754f299e073cf720ef4881': {
      "id": "b6b0c80f07754f299e073cf720ef4881",
      "exchange_id": "paper",
      "exchange_order_id": null,
      "asset": "LTC/BTC",
      "price": 0.1,
      "quantity": 1.0,
      "filled_quantity": 0.0,
      "order_type": "LIMIT_SELL",
      "status": "CREATED",
      "created_time": "2018-01-15T01:16:07.063149",
      "opened_time": null,
      "filled_time": null,
      "canceled_time": null,
      "fee": {},
      "retries": 0
  }}, {
     "BTC": {
         "fr

In [32]:
resp = order_manager.place_order(exchange, orders[o1.id])
orders[o1.id] = resp
orders,exchange.balance

({'5b21ac9d066a4f81ba88f03421cef343': {
      "id": "5b21ac9d066a4f81ba88f03421cef343",
      "exchange_id": "paper",
      "exchange_order_id": "6ca5128457464745a373f3d0a219ca26",
      "asset": "LTC/BTC",
      "price": 0.1,
      "quantity": 1.0,
      "filled_quantity": 0.0,
      "order_type": "LIMIT_BUY",
      "status": "FILLED",
      "created_time": "2018-01-15T01:16:10.437911",
      "opened_time": null,
      "filled_time": null,
      "canceled_time": null,
      "fee": {},
      "retries": 0
  }, 'b6b0c80f07754f299e073cf720ef4881': {
      "id": "b6b0c80f07754f299e073cf720ef4881",
      "exchange_id": "paper",
      "exchange_order_id": null,
      "asset": "LTC/BTC",
      "price": 0.1,
      "quantity": 1.0,
      "filled_quantity": 0.0,
      "order_type": "LIMIT_SELL",
      "status": "CREATED",
      "created_time": "2018-01-15T01:16:07.063149",
      "opened_time": null,
      "filled_time": null,
      "canceled_time": null,
      "fee": {},
      "retries": 0
  }},

In [33]:
resp = order_manager.place_order(exchange, orders[o2.id])
orders[o2.id] = resp
orders,exchange.balance

({'5b21ac9d066a4f81ba88f03421cef343': {
      "id": "5b21ac9d066a4f81ba88f03421cef343",
      "exchange_id": "paper",
      "exchange_order_id": "6ca5128457464745a373f3d0a219ca26",
      "asset": "LTC/BTC",
      "price": 0.1,
      "quantity": 1.0,
      "filled_quantity": 0.0,
      "order_type": "LIMIT_BUY",
      "status": "FILLED",
      "created_time": "2018-01-15T01:16:10.437911",
      "opened_time": null,
      "filled_time": null,
      "canceled_time": null,
      "fee": {},
      "retries": 0
  }, 'b6b0c80f07754f299e073cf720ef4881': {
      "id": "b6b0c80f07754f299e073cf720ef4881",
      "exchange_id": "paper",
      "exchange_order_id": "6b225c5baf8243ff9dc266f3d298d33b",
      "asset": "LTC/BTC",
      "price": 0.1,
      "quantity": 1.0,
      "filled_quantity": 0.0,
      "order_type": "LIMIT_SELL",
      "status": "FILLED",
      "created_time": "2018-01-15T01:16:13.543332",
      "opened_time": null,
      "filled_time": null,
      "canceled_time": null,
      "fee":

In [34]:
limit_buy = order_manager.build_limit_buy_order(exchange, asset, price=.1, quantity=1.0)
limit_sell = order_manager.build_limit_sell_order(exchange, asset, price=.1, quantity=1.0)
market_buy = order_manager.build_market_buy_order(exchange, asset, quantity=1.0)
market_sell = order_manager.build_market_sell_order(exchange, asset, quantity=1.0)
orders = [limit_buy, limit_sell, market_buy, market_sell]
orders

[{
     "id": "32643427b9eb4fa08edd1356afd7f626",
     "exchange_id": "paper",
     "exchange_order_id": null,
     "asset": "LTC/BTC",
     "price": 0.1,
     "quantity": 1.0,
     "filled_quantity": 0.0,
     "order_type": "LIMIT_BUY",
     "status": "CREATED",
     "created_time": "2018-01-15T01:16:15.705996",
     "opened_time": null,
     "filled_time": null,
     "canceled_time": null,
     "fee": {},
     "retries": 0
 }, {
     "id": "fa880fc1d1044897aa0f91bed46df0bc",
     "exchange_id": "paper",
     "exchange_order_id": null,
     "asset": "LTC/BTC",
     "price": 0.1,
     "quantity": 1.0,
     "filled_quantity": 0.0,
     "order_type": "LIMIT_SELL",
     "status": "CREATED",
     "created_time": "2018-01-15T01:16:15.706054",
     "opened_time": null,
     "filled_time": null,
     "canceled_time": null,
     "fee": {},
     "retries": 0
 }, {
     "id": "c9f94886d45c432c95a911895b9f70fb",
     "exchange_id": "paper",
     "exchange_order_id": null,
     "asset": "LTC/BTC",

In [35]:
results = []
for order in orders:
    res = order_manager.place_order(exchange, order)
    results.append(res)
results

ExchangeNotAvailable: binance GET https://api.binance.com/api/v1/exchangeInfo HTTPSConnectionPool(host='api.binance.com', port=443): Max retries exceeded with url: /api/v1/exchangeInfo (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x116a61898>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',)) 

In [36]:
exchange.balance

{
    "BTC": {
        "free": 5.0,
        "used": 0.0,
        "total": 5.0
    },
    "LTC": {
        "free": 0.0,
        "used": 0.0,
        "total": 0.0
    },
    "free": {
        "BTC": 5.0,
        "LTC": 0.0
    },
    "used": {
        "BTC": 0.0,
        "LTC": 0.0
    },
    "total": {
        "BTC": 5.0,
        "LTC": 0.0
    }
}

In [ ]:
exchange.orders

In [37]:
#5e5b1bbbcc9e4edcbca2ec6c0663a1b4
exchange.fetch_orders(asset)

[{'asset': 'LTC/BTC',
  'created_time': '2018-01-15T01:16:10.437911',
  'fee': {},
  'filled': 0.0,
  'id': '6ca5128457464745a373f3d0a219ca26',
  'price': 0.1,
  'quantity': 1.0,
  'side': 'buy',
  'status': 'FILLED',
  'type': 'limit'},
 {'asset': 'LTC/BTC',
  'created_time': '2018-01-15T01:16:13.543332',
  'fee': {},
  'filled': 0.0,
  'id': '6b225c5baf8243ff9dc266f3d298d33b',
  'price': 0.1,
  'quantity': 1.0,
  'side': 'sell',
  'status': 'FILLED',
  'type': 'limit'},
 {'asset': 'LTC/BTC',
  'created_time': '2018-01-15T01:16:17.587446',
  'fee': {},
  'filled': 0.0,
  'id': '4c4f0e4d69584e35bb4cc6efb0a47d61',
  'price': 0.1,
  'quantity': 1.0,
  'side': 'buy',
  'status': 'FILLED',
  'type': 'limit'},
 {'asset': 'LTC/BTC',
  'created_time': '2018-01-15T01:16:17.587728',
  'fee': {},
  'filled': 0.0,
  'id': 'df8a97dd577a4578ac57dd54c295d266',
  'price': 0.1,
  'quantity': 1.0,
  'side': 'sell',
  'status': 'FILLED',
  'type': 'limit'}]

In [ ]:
exchange.fetch_order('57eb982b9afb49208b081a117022ffbe')

In [ ]:
updated_orders = exchange.fetch_orders(asset)
ex_order_ids = [order['id'] for order in updated_orders]
print(ex_order_ids)
updated_orders

In [ ]:
ex_orders = order_manager.get_orders(exchange, ex_order_ids)
print(ex_orders)

In [ ]:
type(ex_orders[0])

In [ ]:
print("PENDING", order_manager.get_pending_orders(ex_orders))
print("CANCELED", order_manager.get_canceled_orders(ex_orders))
print("FILLED", order_manager.get_filled_orders(ex_orders))

### Position

In [38]:
asset = Asset(c.LTC, c.BTC)
pos = Position(asset, quantity=1, cost_price=250.0)
print("Asset", pos.asset.to_dict())
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)

pos.update(1, 200)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)

pos.update(-1, 200)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)

# Go short
pos.update(-2, 150)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)

# Close the short for a LOSS and go long (price went up, we had to pay $300 to cover our short)
# then we bought another share because we're newbs
pos.update(2, 300)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)

Asset {'id': 'LTC_BTC', 'symbol': 'LTC/BTC', 'base': 'LTC', 'quote': 'BTC'}
Value 250.0 Quantity 1 Cost 250.0
Value 450.0 Quantity 2 Cost 225.0
Value 225.0 Quantity 1 Cost 225.0
Value -150 Quantity -1 Cost 150
Value 300 Quantity 1 Cost 300


In [39]:
"""
Assume that an investor made the following consecutive fund purchases in a taxable account: 1,500 shares at $20, 1,000 shares at $10 and 1,250 shares at $8. The investor’s average cost basis is calculated by dividing $50,000/3,750 shares. The average cost is $13.33.

Suppose the investor then sells 1,000 shares of the fund at $19.

Gain/loss using average cost basis: ($19 - $13.33) x 1,000 shares = $5,667
"""

asset = Asset(c.LTC, c.BTC)
pos = Position(asset, quantity=1500, cost_price=20.0)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)
pos.update(txn_quantity=1000, txn_price=10)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)
pos.update(txn_quantity=1250, txn_price=8)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)

last_cost = pos.cost_price
pos.update(txn_quantity=-1000, txn_price=19)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)
print("Profit", (19 - last_cost) * 1000)

Value 30000.0 Quantity 1500 Cost 20.0
Value 40000.0 Quantity 2500 Cost 16.0
Value 50000.0 Quantity 3750 Cost 13.333333333333334
Value 36666.66666666667 Quantity 2750 Cost 13.333333333333334
Profit 5666.666666666666


In [40]:
Position.from_dict(pos.to_dict()).to_dict()

{'asset': 'LTC/BTC',
 'cost_price': 13.333333333333334,
 'latest_price': 19,
 'quantity': 2750}

### PerformanceTracker

In [41]:
asset = Asset(c.BTC, c.USD)
perf = PerformanceTracker(starting_cash=5000, timeframe=Timeframe.ONE_MIN, store=None)
perf.to_dict()

{'periods': [],
 'pnl': 0.0,
 'returns': 0.0,
 'starting_cash': 5000,
 'timeframe': 'ONE_MIN'}

In [42]:
# Buy 1 BTC for $1000
pos1 = Position(asset, quantity=1, cost_price=1000.0)
positions = [pos1]
perf.add_period(
    start=datetime.datetime.utcnow(),
    cash=4000.0,
    positions=positions
)
perf.to_dict()

{'periods': [{'end_cash': 4000.0,
   'end_time': '2018-01-15T01:18:21.977029',
   'end_value': 5000.0,
   'pnl': 0.0,
   'returns': 0.0,
   'start_cash': 5000,
   'start_time': '2018-01-15T01:17:21.977029',
   'start_value': 5000}],
 'pnl': 0.0,
 'returns': 0.0,
 'starting_cash': 5000,
 'timeframe': 'ONE_MIN'}

In [43]:
"""
Value of BTC increased $100
Position Return
    Return = .10
    PnL = $100
Cumulative Return
    Return = $100 / $5000 = .02
    PnL = $100
"""
pos1.latest_price = 1100
perf.add_period(
    start=datetime.datetime.utcnow(),
    cash=4000.0,
    positions=positions
)
perf.to_dict()

{'periods': [{'end_cash': 4000.0,
   'end_time': '2018-01-15T01:18:21.977029',
   'end_value': 5000.0,
   'pnl': 0.0,
   'returns': 0.0,
   'start_cash': 5000,
   'start_time': '2018-01-15T01:17:21.977029',
   'start_value': 5000},
  {'end_cash': 4000.0,
   'end_time': '2018-01-15T01:18:24.146686',
   'end_value': 5100.0,
   'pnl': 100.0,
   'returns': 0.02,
   'start_cash': 4000.0,
   'start_time': '2018-01-15T01:17:24.146686',
   'start_value': 5000.0}],
 'pnl': 100.0,
 'returns': 0.02,
 'starting_cash': 5000,
 'timeframe': 'ONE_MIN'}

In [44]:
"""
Value of BTC increased again $400
Position Return
    Return = .5
    PnL = $500
Cumulative Return
    Return = $500 / $5000 = .1
    PnL = $500
"""
pos1.latest_price = 1500
perf.add_period(
    start=datetime.datetime.utcnow(),
    cash=4000.0,
    positions=positions
)
perf.to_dict()

{'periods': [{'end_cash': 4000.0,
   'end_time': '2018-01-15T01:18:21.977029',
   'end_value': 5000.0,
   'pnl': 0.0,
   'returns': 0.0,
   'start_cash': 5000,
   'start_time': '2018-01-15T01:17:21.977029',
   'start_value': 5000},
  {'end_cash': 4000.0,
   'end_time': '2018-01-15T01:18:24.146686',
   'end_value': 5100.0,
   'pnl': 100.0,
   'returns': 0.02,
   'start_cash': 4000.0,
   'start_time': '2018-01-15T01:17:24.146686',
   'start_value': 5000.0},
  {'end_cash': 4000.0,
   'end_time': '2018-01-15T01:18:26.317650',
   'end_value': 5500.0,
   'pnl': 500.0,
   'returns': 0.1,
   'start_cash': 4000.0,
   'start_time': '2018-01-15T01:17:26.317650',
   'start_value': 5100.0}],
 'pnl': 500.0,
 'returns': 0.1,
 'starting_cash': 5000,
 'timeframe': 'ONE_MIN'}

In [45]:
# Buy 1 ETH for $500
asset = Asset(c.ETH, c.USD)
pos2 = Position(asset, quantity=1, cost_price=500.0)
positions.append(pos2)
perf.add_period(
    start=datetime.datetime.utcnow(),
    cash=3500.0,
    positions=positions
)
perf.to_dict()

{'periods': [{'end_cash': 4000.0,
   'end_time': '2018-01-15T01:18:21.977029',
   'end_value': 5000.0,
   'pnl': 0.0,
   'returns': 0.0,
   'start_cash': 5000,
   'start_time': '2018-01-15T01:17:21.977029',
   'start_value': 5000},
  {'end_cash': 4000.0,
   'end_time': '2018-01-15T01:18:24.146686',
   'end_value': 5100.0,
   'pnl': 100.0,
   'returns': 0.02,
   'start_cash': 4000.0,
   'start_time': '2018-01-15T01:17:24.146686',
   'start_value': 5000.0},
  {'end_cash': 4000.0,
   'end_time': '2018-01-15T01:18:26.317650',
   'end_value': 5500.0,
   'pnl': 500.0,
   'returns': 0.1,
   'start_cash': 4000.0,
   'start_time': '2018-01-15T01:17:26.317650',
   'start_value': 5100.0},
  {'end_cash': 3500.0,
   'end_time': '2018-01-15T01:18:28.269421',
   'end_value': 5500.0,
   'pnl': 500.0,
   'returns': 0.1,
   'start_cash': 4000.0,
   'start_time': '2018-01-15T01:17:28.269421',
   'start_value': 5500.0}],
 'pnl': 500.0,
 'returns': 0.1,
 'starting_cash': 5000,
 'timeframe': 'ONE_MIN'}

In [46]:
"""
Value of ETH decreased $100
Position Return
    Return = -100/500
    PnL = -100
Cumulative Return
    Return = 400 / 5000 = .08
    PnL = 400
"""
pos2.latest_price = 400
perf.add_period(
    start=datetime.datetime.utcnow(),
    cash=3500.0,
    positions=positions
)
perf.to_dict()

{'periods': [{'end_cash': 4000.0,
   'end_time': '2018-01-15T01:18:21.977029',
   'end_value': 5000.0,
   'pnl': 0.0,
   'returns': 0.0,
   'start_cash': 5000,
   'start_time': '2018-01-15T01:17:21.977029',
   'start_value': 5000},
  {'end_cash': 4000.0,
   'end_time': '2018-01-15T01:18:24.146686',
   'end_value': 5100.0,
   'pnl': 100.0,
   'returns': 0.02,
   'start_cash': 4000.0,
   'start_time': '2018-01-15T01:17:24.146686',
   'start_value': 5000.0},
  {'end_cash': 4000.0,
   'end_time': '2018-01-15T01:18:26.317650',
   'end_value': 5500.0,
   'pnl': 500.0,
   'returns': 0.1,
   'start_cash': 4000.0,
   'start_time': '2018-01-15T01:17:26.317650',
   'start_value': 5100.0},
  {'end_cash': 3500.0,
   'end_time': '2018-01-15T01:18:28.269421',
   'end_value': 5500.0,
   'pnl': 500.0,
   'returns': 0.1,
   'start_cash': 4000.0,
   'start_time': '2018-01-15T01:17:28.269421',
   'start_value': 5500.0},
  {'end_cash': 3500.0,
   'end_time': '2018-01-15T01:18:30.325291',
   'end_value': 54

In [47]:
PerformanceTracker.from_dict(perf.to_dict())

{
    "starting_cash": 5000,
    "timeframe": "ONE_MIN",
    "periods": [
        {
            "start_time": "2018-01-15T01:17:21.977029",
            "end_time": "2018-01-15T01:18:21.977029",
            "start_cash": 5000,
            "end_cash": 4000.0,
            "start_value": 5000,
            "end_value": 5000.0,
            "pnl": 0.0,
            "returns": 0.0
        },
        {
            "start_time": "2018-01-15T01:17:24.146686",
            "end_time": "2018-01-15T01:18:24.146686",
            "start_cash": 4000.0,
            "end_cash": 4000.0,
            "start_value": 5000.0,
            "end_value": 5100.0,
            "pnl": 100.0,
            "returns": 0.02
        },
        {
            "start_time": "2018-01-15T01:17:26.317650",
            "end_time": "2018-01-15T01:18:26.317650",
            "start_cash": 4000.0,
            "end_cash": 4000.0,
            "start_value": 5100.0,
            "end_value": 5500.0,
            "pnl": 500.0,
            "re

### Portfolio

In [48]:
starting_cash = 5000
exchange = load_exchange(c.PAPER)
perf = PerformanceTracker(starting_cash, Timeframe.ONE_MIN, store=None)
portfolio = Portfolio(starting_cash, perf)

{'BTC': {'free': 1.0, 'used': 0.0, 'total': 1.0}, 'free': {'BTC': 1.0}, 'used': {'BTC': 0.0}, 'total': {'BTC': 1.0}}


In [49]:
# Buy 1 BTC for $1000
asset = Asset(c.BTC, c.USD)
order = Order(
    exchange_id=exchange.id, 
    asset=asset,
    price=1000., 
    quantity=1, 
    order_type=OrderType.LIMIT_BUY
)
order

{
    "id": "4a77501106e546ee8de3e075fa3fa919",
    "exchange_id": "paper",
    "exchange_order_id": null,
    "asset": "BTC/USD",
    "price": 1000.0,
    "quantity": 1,
    "filled_quantity": 0.0,
    "order_type": "LIMIT_BUY",
    "status": "CREATED",
    "created_time": "2018-01-15T01:17:37.000497",
    "opened_time": null,
    "filled_time": null,
    "canceled_time": null,
    "fee": {},
    "retries": 0
}

In [50]:
portfolio.update(filled_orders=[order])
portfolio

{
    "starting_cash": 5000,
    "cash": 4000.0,
    "weights": {
        "BTC_USD": 0.2,
        "cash": 0.8
    },
    "pnl": 0.0,
    "returns": 0.0,
    "total_value": 5000.0,
    "performance": {
        "starting_cash": 5000,
        "timeframe": "ONE_MIN",
        "periods": [
            {
                "start_time": "2018-01-15T01:17:38.865567",
                "end_time": "2018-01-15T01:18:38.865567",
                "start_cash": 5000,
                "end_cash": 4000.0,
                "start_value": 5000,
                "end_value": 5000.0,
                "pnl": 0.0,
                "returns": 0.0
            }
        ],
        "pnl": 0.0,
        "returns": 0.0
    },
    "positions": [
        {
            "asset": "BTC/USD",
            "quantity": 1,
            "cost_price": 1000.0,
            "latest_price": 1000.0
        }
    ]
}

In [51]:
# Buy more BTC, price has risen $500
order = Order(
    exchange_id=exchange.id, 
    asset=asset,
    price=1500., 
    quantity=1, 
    order_type=OrderType.LIMIT_BUY
)
portfolio.update(filled_orders=[order])
portfolio

{
    "starting_cash": 5000,
    "cash": 2500.0,
    "weights": {
        "BTC_USD": 0.5454545454545454,
        "cash": 0.45454545454545453
    },
    "pnl": 500.0,
    "returns": 0.1,
    "total_value": 5500.0,
    "performance": {
        "starting_cash": 5000,
        "timeframe": "ONE_MIN",
        "periods": [
            {
                "start_time": "2018-01-15T01:17:38.865567",
                "end_time": "2018-01-15T01:18:38.865567",
                "start_cash": 5000,
                "end_cash": 4000.0,
                "start_value": 5000,
                "end_value": 5000.0,
                "pnl": 0.0,
                "returns": 0.0
            },
            {
                "start_time": "2018-01-15T01:17:40.917121",
                "end_time": "2018-01-15T01:18:40.917121",
                "start_cash": 4000.0,
                "end_cash": 2500.0,
                "start_value": 5000.0,
                "end_value": 5500.0,
                "pnl": 500.0,
                "r

In [52]:
# Sell all BTC at profit
order = Order(
    exchange_id=exchange.id, 
    asset=asset,
    price=1500., 
    quantity=2, 
    order_type=OrderType.LIMIT_SELL
)
portfolio.update(filled_orders=[order])
portfolio

{
    "starting_cash": 5000,
    "cash": 5500.0,
    "weights": {
        "BTC_USD": 0.0,
        "cash": 1.0
    },
    "pnl": 500.0,
    "returns": 0.1,
    "total_value": 5500.0,
    "performance": {
        "starting_cash": 5000,
        "timeframe": "ONE_MIN",
        "periods": [
            {
                "start_time": "2018-01-15T01:17:38.865567",
                "end_time": "2018-01-15T01:18:38.865567",
                "start_cash": 5000,
                "end_cash": 4000.0,
                "start_value": 5000,
                "end_value": 5000.0,
                "pnl": 0.0,
                "returns": 0.0
            },
            {
                "start_time": "2018-01-15T01:17:40.917121",
                "end_time": "2018-01-15T01:18:40.917121",
                "start_cash": 4000.0,
                "end_cash": 2500.0,
                "start_value": 5000.0,
                "end_value": 5500.0,
                "pnl": 500.0,
                "returns": 0.1
            },
   

In [53]:
Portfolio.from_dict(portfolio.to_dict())

{
    "starting_cash": 5000,
    "cash": 5500.0,
    "weights": {
        "BTC_USD": 0.0,
        "cash": 1.0
    },
    "pnl": 500.0,
    "returns": 0.1,
    "total_value": 5500.0,
    "performance": {
        "starting_cash": 5000,
        "timeframe": "ONE_MIN",
        "periods": [
            {
                "start_time": "2018-01-15T01:17:38.865567",
                "end_time": "2018-01-15T01:18:38.865567",
                "start_cash": 5000,
                "end_cash": 4000.0,
                "start_value": 5000,
                "end_value": 5000.0,
                "pnl": 0.0,
                "returns": 0.0
            },
            {
                "start_time": "2018-01-15T01:17:40.917121",
                "end_time": "2018-01-15T01:18:40.917121",
                "start_cash": 4000.0,
                "end_cash": 2500.0,
                "start_value": 5000.0,
                "end_value": 5500.0,
                "pnl": 500.0,
                "returns": 0.1
            },
   

### Record

In [54]:
feed = get_test_csv_feed(c.PAPER)

metrics = {
    'SMA': [1, 2, 3, 4],
    'RSI': [3, 4, 2, 1]
}

o1 = Order(c.PAPER, Asset(c.ETH, c.BTC), 5.0, 100.0, OrderType.LIMIT_BUY)
o2 = Order(c.PAPER, Asset(c.LTC, c.BTC), 5.0, 100.0, OrderType.LIMIT_BUY)
orders = {
    o1.id: o1,
    o2.id: o2
}

/Users/cfortuner/Dropbox/punisher/paper_ETH_BTC_30m.csv
{'BTC': {'free': 1.0, 'used': 0.0, 'total': 1.0}, 'free': {'BTC': 1.0}, 'used': {'BTC': 0.0}, 'total': {'BTC': 1.0}}
Downloading: ETH/BTC


ExchangeNotAvailable: binance GET https://api.binance.com/api/v1/exchangeInfo HTTPSConnectionPool(host='api.binance.com', port=443): Max retries exceeded with url: /api/v1/exchangeInfo (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x116add4e0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',)) 

In [ ]:
root_dir = os.path.join(cfg.DATA_DIR, 'default')
store = FileStore(root_dir)

record = Record(config={}, portfolio=portfolio, balance=Balance(), store=store)
record.orders = orders
record.metrics = metrics
record.ohlcv = feed.history()
record.save()
record = Record.load(root_dir)

In [ ]:
print(record.config)
print(record.metrics)
print(record.balance)
print(record.orders)
print(record.ohlcv.head()[['time_utc','close']])
print(record.portfolio)

### Context

In [55]:
feed = get_test_csv_feed(c.PAPER)
ctx = Context(DEFAULT_CONFIG, exchange, feed, record)

/Users/cfortuner/Dropbox/punisher/paper_ETH_BTC_30m.csv
{'BTC': {'free': 1.0, 'used': 0.0, 'total': 1.0}, 'free': {'BTC': 1.0}, 'used': {'BTC': 0.0}, 'total': {'BTC': 1.0}}
Downloading: ETH/BTC


ExchangeNotAvailable: binance GET https://api.binance.com/api/v1/exchangeInfo HTTPSConnectionPool(host='api.binance.com', port=443): Max retries exceeded with url: /api/v1/exchangeInfo (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x116aef8d0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',)) 

In [ ]:
ctx = Context.from_config(DEFAULT_CONFIG)
print(ctx.record.config)
print(ctx.record.metrics)
print(ctx.record.balance)
print(ctx.record.orders)
print(ctx.record.ohlcv)
print(ctx.record.portfolio)

### Strategy

In [57]:
# https://www.backtrader.com/docu/quickstart/quickstart.html
# https://enigmampc.github.io/catalyst/beginner-tutorial.html#basics
from punisher.strategies.simple import SimpleStrategy
from punisher.trading import order_manager
strategy = SimpleStrategy()

In [69]:
import shutil
from punisher.data.store import DATA_STORES, FILE_STORE
from punisher.data.feed import EXCHANGE_FEED, CSV_FEED
from punisher.trading.context import default_config, TradingMode

config = default_config(TradingMode.BACKTEST)

shutil.rmtree(os.path.join(cfg.DATA_DIR, config['experiment']), ignore_errors=True)
ctx = Context.from_config(config)

Loading feed: /Users/cfortuner/Dropbox/punisher/paper_ETH_BTC_30m.csv


In [75]:
orders = strategy.process(ctx.feed.next(), ctx)

---------------------------------------
Epoch 4 - Timestep: 2018-01-09T01:30:00
---------------------------------------
OHLCV
    O: 0.0771 | C: 0.0764 | V: 14292.4 | T: 2018-01-09T01:30:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0764 | Quantity: 1.0000
PERFORMANCE
    Cash: 1.0000 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 1.0, 'used': 0.0, 'total': 1.0}
POSITIONS
METRICS
    SMA: 5.00
    RSI: 10.00


In [76]:
resp = order_manager.place_orders(ctx.exchange, orders)
resp

[{
     "id": "f5c54980a7b145618428173ed16129aa",
     "exchange_id": "paper",
     "exchange_order_id": "74114d59371640329e132c7a7b8eb188",
     "asset": "ETH/BTC",
     "price": 0.076405,
     "quantity": 1.0,
     "filled_quantity": 0.0,
     "order_type": "LIMIT_BUY",
     "status": "FILLED",
     "created_time": "2018-01-15T01:31:21.762111",
     "opened_time": null,
     "filled_time": null,
     "canceled_time": null,
     "fee": {},
     "retries": 0
 }]

In [77]:
# We're not updating the virtual balance, only the exchange 
# (which is fine until we want to have a multi-exchange algo)
print(ctx.exchange.balance)

{
    "BTC": {
        "free": 0.9235949999999999,
        "used": 0.0,
        "total": 0.9235949999999999
    },
    "ETH": {
        "free": 1.0,
        "used": 0.0,
        "total": 1.0
    },
    "free": {
        "BTC": 0.9235949999999999,
        "ETH": 1.0
    },
    "used": {
        "BTC": 0.0,
        "ETH": 0.0
    },
    "total": {
        "BTC": 0.9235949999999999,
        "ETH": 1.0
    }
}


### Runner

In [ ]:
import shutil
from punisher.trading.runner import *

mystrategy = SimpleStrategy()
runner.backtest("jupyter-run", mystrategy)

Starting backtest ...
{'experiment': 'default', 'cash_asset': 'BTC', 'starting_cash': 1.0, 'store': 'CSV_STORE', 'balance': {'BTC': {'free': 1.0, 'used': 0.0, 'total': 1.0}, 'free': {'BTC': 1.0}, 'used': {'BTC': 0.0}, 'total': {'BTC': 1.0}}, 'feed': {'fpath': '/Users/cfortuner/Dropbox/punisher/paper_ETH_BTC_30m.csv', 'symbols': ['ETH/BTC'], 'timeframe': 'THIRTY_MIN', 'name': 'CSV_FEED', 'start': '2018-01-01T00:00:00'}, 'exchange': {'exchange_id': 'paper'}}
---------------------------------------
Epoch 1 - Timestep: 2018-01-09T00:00:00
---------------------------------------
OHLCV
    O: 0.0760 | C: 0.0766 | V: 7900.8 | T: 2018-01-09T00:00:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0766 | Quantity: 1.0000
PERFORMANCE
    Cash: 1.0000 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 1.0, 'used': 0.0, 'total': 1.0}
POSITIONS
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 2 - Timestep: 2018-01-09T00:30:00
--------------------

Loading feed: /Users/cfortuner/Dropbox/punisher/paper_ETH_BTC_30m.csv


METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 4 - Timestep: 2018-01-09T01:30:00
---------------------------------------
OHLCV
    O: 0.0771 | C: 0.0764 | V: 14292.4 | T: 2018-01-09T01:30:00
ORDERS
    1: ETH/BTC | LIMIT_SELL | Price: 0.0764 | Quantity: 1.0000
PERFORMANCE
    Cash: 0.9238 Total Val: 1.0008 PnL: 0.0008 Returns: 0.0008
BALANCE
    BTC - {'free': 0.84742899999999999, 'used': 0.0, 'total': 0.84742899999999999}
    ETH - {'free': 2.0, 'used': 0.0, 'total': 2.0}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 1.0, 'cost_price': 0.076923000000000005, 'latest_price': 0.076923000000000005}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 5 - Timestep: 2018-01-09T02:00:00
---------------------------------------
OHLCV
    O: 0.0765 | C: 0.0773 | V: 10995.4 | T: 2018-01-09T02:00:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0773 | Quantity: 1.0000
PERFORMANCE
    Cash: 1.0002 Total Val: 1.0002 PnL: 0.0002 Retu

Balance is not sufficient to create order!
Balance is not sufficient to create order!


---------------------------------------
Epoch 16 - Timestep: 2018-01-09T07:30:00
---------------------------------------
OHLCV
    O: 0.0799 | C: 0.0805 | V: 6801.3 | T: 2018-01-09T07:30:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0805 | Quantity: 1.0000
PERFORMANCE
    Cash: 1.0798 Total Val: 0.9998 PnL: -0.0002 Returns: -0.0002
BALANCE
    BTC - {'free': 1.0033779999999999, 'used': 0.0, 'total': 1.0033779999999999}
    ETH - {'free': 0.0, 'used': 0.0, 'total': 0.0}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': -1.0, 'cost_price': 0.080004999999999993, 'latest_price': 0.080004999999999993}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 17 - Timestep: 2018-01-09T08:00:00
---------------------------------------
OHLCV
    O: 0.0805 | C: 0.0812 | V: 9038.7 | T: 2018-01-09T08:00:00
ORDERS
    1: ETH/BTC | LIMIT_SELL | Price: 0.0812 | Quantity: 1.0000
PERFORMANCE
    Cash: 0.9993 Total Val: 0.9993 PnL: -0.0007 Returns: -0.0007
BALANCE
    BTC - {'fr

...

---------------------------------------
Epoch 118 - Timestep: 2018-01-11T10:30:00
---------------------------------------
OHLCV
    O: 0.0901 | C: 0.0892 | V: 5058.8 | T: 2018-01-11T10:30:00
ORDERS
    1: ETH/BTC | LIMIT_SELL | Price: 0.0892 | Quantity: 1.0000
PERFORMANCE
    Cash: 1.1032 Total Val: 1.0134 PnL: 0.0134 Returns: 0.0134
BALANCE
    BTC - {'free': 1.0267739999999994, 'used': 0.0, 'total': 1.0267739999999994}
    ETH - {'free': 0.0, 'used': 0.0, 'total': 0.0}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': -1.0, 'cost_price': 0.089744000000000004, 'latest_price': 0.089744000000000004}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 119 - Timestep: 2018-01-11T11:00:00
---------------------------------------
OHLCV
    O: 0.0894 | C: 0.0878 | V: 7604.3 | T: 2018-01-11T11:00:00
ORDERS
    1: ETH/BTC | LIMIT_SELL | Price: 0.0878 | Quantity: 1.0000
PERFORMANCE
    Cash: 1.1032 Total Val: 1.0134 PnL: 0.0134 Returns: 0.0134
BALANCE
    BTC - {'fre

Balance is not sufficient to create order!
Balance is not sufficient to create order!
Balance is not sufficient to create order!


    BTC - {'free': 1.0267739999999994, 'used': 0.0, 'total': 1.0267739999999994}
    ETH - {'free': 0.0, 'used': 0.0, 'total': 0.0}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': -1.0, 'cost_price': 0.089744000000000004, 'latest_price': 0.089744000000000004}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 122 - Timestep: 2018-01-11T12:30:00
---------------------------------------
OHLCV
    O: 0.0880 | C: 0.0875 | V: 5546.5 | T: 2018-01-11T12:30:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0875 | Quantity: 1.0000
PERFORMANCE
    Cash: 1.0152 Total Val: 1.0152 PnL: 0.0152 Returns: 0.0152
BALANCE
    BTC - {'free': 0.93877499999999936, 'used': 0.0, 'total': 0.93877499999999936}
    ETH - {'free': 1.0, 'used': 0.0, 'total': 1.0}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.0, 'cost_price': 0.0, 'latest_price': 0.087999000000000008}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 123 - Timestep: 2018-01-11T13:00:

PERFORMANCE
    Cash: 0.6656 Total Val: 1.0227 PnL: 0.0227 Returns: 0.0227
BALANCE
    BTC - {'free': 0.58919199999999938, 'used': 0.0, 'total': 0.58919199999999938}
    ETH - {'free': 5.0, 'used': 0.0, 'total': 5.0}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 4.0, 'cost_price': 0.088081237037037033, 'latest_price': 0.089287000000000005}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 135 - Timestep: 2018-01-11T19:00:00
---------------------------------------
OHLCV
    O: 0.0891 | C: 0.0877 | V: 8253.6 | T: 2018-01-11T19:00:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0877 | Quantity: 1.0000
PERFORMANCE
    Cash: 0.7547 Total Val: 1.0221 PnL: 0.0221 Returns: 0.0221
BALANCE
    BTC - {'free': 0.67831399999999942, 'used': 0.0, 'total': 0.67831399999999942}
    ETH - {'free': 4.0, 'used': 0.0, 'total': 4.0}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 3.0, 'cost_price': 0.088081237037037033, 'latest_price': 0.089122000000000007}
METRICS
    SMA

No data after prior poll: 2018-01-11 23:30:00
No data after prior poll: 2018-01-11 23:30:00
No data after prior poll: 2018-01-11 23:30:00
No data after prior poll: 2018-01-11 23:30:00
No data after prior poll: 2018-01-11 23:30:00
No data after prior poll: 2018-01-11 23:30:00
No data after prior poll: 2018-01-11 23:30:00
Balance is not sufficient to create order!
No data after prior poll: 2018-01-11 23:30:00
No data after prior poll: 2018-01-11 23:30:00
No data after prior poll: 2018-01-11 23:30:00
No data after prior poll: 2018-01-11 23:30:00
No data after prior poll: 2018-01-11 23:30:00
No data after prior poll: 2018-01-11 23:30:00
No data after prior poll: 2018-01-11 23:30:00
No data after prior poll: 2018-01-11 23:30:00
No data after prior poll: 2018-01-11 23:30:00
No data after prior poll: 2018-01-11 23:30:00
No data after prior poll: 2018-01-11 23:30:00
No data after prior poll: 2018-01-11 23:30:00
No data after prior poll: 2018-01-11 23:30:00
No data after prior poll: 2018-01-11 

### ChartDataProviders

In [ ]:
import threading
from utils.dates import date_to_str

root = os.path.join(cfg.DATA_DIR, 'default')
rp = RecordChartDataProvider(root)

In [ ]:
# TODO
print("OHLCV\n", rp.get_ohlcv())
print("\nPERFORMANCE\n", rp.get_performance())
print("\nRETURNS\n", rp.get_returns())
print("\nPNL\n", rp.get_pnl())

print("\nBALANCE\n", rp.get_balance())
print("\nBALANCE_DICT\n", rp.get_balance_dct())
print("\nPOSITIONS\n", rp.get_positions())
print("\nPOSITIONS_DICT\n", rp.get_positions_dct())

print("\nORDERS\n", rp.get_orders())
print("\nORDERS_DICT\n", rp.get_orders_dct())
print("\nMETRICS\n", rp.get_metrics())

### Charts

In [ ]:
feed = get_test_csv_feed(c.PAPER)
start = feed.history().iloc[0]['time_utc']
end = feed.history().iloc[-1]['time_utc']

In [ ]:
# Basic plot
utils.charts.plot_range(feed.history(), start, end, 'close')

### Dash

In [ ]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

periods = record.portfolio.perf.periods
df = pd.DataFrame([
    [p['end_time'], p['pnl']] for p in periods
], columns=['time_utc','pnl'])
df

In [ ]:
positions = pd.DataFrame([p.to_dict() for p in record.portfolio.positions])
dct = [p.to_dict() for p in record.portfolio.positions]
{p['asset']: p for p in dct}

In [ ]:
cols = ['coin', 'free', 'used', 'total']
coins = b.currencies
dct = b.to_dict()
df = pd.DataFrame(
    [[c, dct[c]['free'], dct[c]['used'], dct[c]['total']] for c in coins],
    columns=cols
)
df